In [1]:
import os
from uuid import uuid1

import cv2
import ipywidgets as widgets
from IPython.display import display
from ipycanvas import Canvas
from ipywidgets import Image


In [2]:
#model_road = torchvision.models.resnet18(weights=None)
#model_road.fc = torch.nn.Linear(512, 2)
#model_road.load_state_dict(torch.load('best_steering_model_xy.pth', map_location=torch.device('cpu')))

In [3]:
#def get_road_direction(image) -> (float, float):
#    model_xy = model_road(image).detach().float().cpu().numpy().flatten()
#    model_x = model_xy[0]
#    model_y = (0.5 - model_xy[1]) / 2.0
#    return model_x, model_y

In [4]:
img_list = []
for root, dirs, files in os.walk("img", topdown=False):
    for name in files:
        img_list.append(root + "/" + name)

In [5]:
counter = 0

btn = widgets.Button(
    description="Next"
)

out = widgets.Output()

In [6]:

DATASET_DIR = 'dataset_xy'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(DATASET_DIR)
except FileExistsError:
    print('Directories not created because they already exist')

In [7]:
#image_widget = ClickableImageWidget(width=224, height=224)
snapshot_widget = widgets.Image(width=224, height=224)

In [8]:

canvas = Canvas(width=224, height=224)




In [9]:

@out.capture()
def on_canvas_click(x,y):
    x = int(x)
    y = int(y)
    print(f"{x} {y}")
    canvas.stroke_style = "#00ff00"
    canvas.stroke_circle(x,y,8)

def on_click():
    global counter
    if len(img_list) > counter:
        canvas.clear()
        out.clear_output()
        image = Image.from_file(img_list[counter])
        image_file = open(img_list[counter], 'rb').read()

        snapshot_widget.value = image_file
        canvas.draw_image(image, 0, 0)
        counter += 1


btn.on_click(on_click)
canvas.on_mouse_down(on_canvas_click)

on_click()

In [10]:
def save_snapshot(_, content, msg):
    global counter
    if content['event'] == 'click':
        with out:
            display("event")
        data = content['eventData']
        x = data['offsetX']
        y = data['offsetY']

        # save to disk
        #dataset.save_entry(category_widget.value, camera.value, x, y)
        uuid = 'xy_%03d_%03d_%s' % (x, y, uuid1())
        image_path = os.path.join(DATASET_DIR, uuid + '.jpg')
        #with open(image_path, 'wb') as f:
        #    f.write(image_widget.value)
        snapshot = open(img_list[counter], 'rb').read()
        snapshot = cv2.circle(snapshot, (x, y), 8, (0, 255, 0), 3)
        snapshot_widget.value = snapshot

#image_widget.on_msg(save_snapshot)

In [11]:
data_collection_widget = widgets.VBox([
    widgets.HBox([canvas, snapshot_widget]),
    btn
])
display(data_collection_widget)
display(out)